<a href="https://colab.research.google.com/github/ymoslem/Speech/blob/main/FLEURS-GA-EN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install datasets -q

In [ ]:
# Load the FLEURS dataset for English and Irish
# Change name="ga_ie" to the language of your choice
# Here we merge the 3 splits "train+validation+test", but you can download them separately

from datasets import load_dataset

fleurs_dataset_en = load_dataset("google/fleurs",
                                  name="en_us",
                                  split="train+validation+test",
                                  token=True,
                                  trust_remote_code=True,
                                )

fleurs_dataset_ga = load_dataset("google/fleurs",
                                  name="ga_ie",
                                  split="train+validation+test",
                                  token=True,
                                  trust_remote_code=True,
                                )

In [ ]:
print(fleurs_dataset_en)
print(fleurs_dataset_ga)

Dataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 3643
})
Dataset({
    features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
    num_rows: 4056
})


In [ ]:
fleurs_dataset_en[0]

{'id': 903,
 'num_samples': 108800,
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/ee72a6213f7cabb647adadc74f3888674aa278435dca4f54892382eda44c8014/10004088536354799741.wav',
 'audio': {'path': 'train/10004088536354799741.wav',
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -3.15904617e-06, -3.03983688e-06, -3.27825546e-06]),
  'sampling_rate': 16000},
 'transcription': 'a tornado is a spinning column of very low-pressure air which sucks the surrounding air inward and upward',
 'raw_transcription': 'A tornado is a spinning column of very low-pressure air, which sucks the surrounding air inward and upward.',
 'gender': 1,
 'lang_id': 19,
 'language': 'English',
 'lang_group_id': 0}

In [ ]:
# Remove unnecessary columns

fleurs_dataset_en = fleurs_dataset_en.remove_columns(['num_samples', 'path', 'audio', 'transcription', 'gender', 'lang_id', 'language', 'lang_group_id'])
fleurs_dataset_en = fleurs_dataset_en.rename_column("raw_transcription", "text_en")

fleurs_dataset_ga = fleurs_dataset_ga.remove_columns(['num_samples', 'path', 'transcription', 'gender', 'lang_id', 'language', 'lang_group_id'])
fleurs_dataset_ga = fleurs_dataset_ga.rename_column("raw_transcription", "text_ga")

print(fleurs_dataset_en)
print(fleurs_dataset_ga)

Dataset({
    features: ['id', 'text_en'],
    num_rows: 3643
})
Dataset({
    features: ['id', 'audio', 'text_ga'],
    num_rows: 4056
})


In [ ]:
fleurs_dataset_ga["audio"][0]

{'path': 'train/10009174761044778838.wav',
 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00030464,
        -0.00026166, -0.00036532]),
 'sampling_rate': 16000}

In [ ]:
# Cast the audio column to Audio with sampling rate 16000

from datasets import Audio

fleurs_dataset_ga = fleurs_dataset_ga.cast_column("audio", Audio(sampling_rate=16000))

print(fleurs_dataset_ga)

Dataset({
    features: ['id', 'audio', 'text_ga'],
    num_rows: 4056
})


In [ ]:
# TEST: Count the number of common examples in the English and Irish datasets

count = 0
for idx in fleurs_dataset_ga["id"]:
  if idx in fleurs_dataset_en["id"]:
    count += 1

print(count)

3991


In [ ]:
# Merge the English and Irish datasets based on the id

def merge_examples(example):
  id = example['id']
  if id in fleurs_dataset_en['id']:
    example["text_en"] = fleurs_dataset_en['text_en'][fleurs_dataset_en['id'].index(id)]
  else:
    example["text_en"] = None

  return example

In [ ]:
# Map the merge function to the Irish dataset
merged_dataset = fleurs_dataset_ga.map(merge_examples)

# Filter out None values
fleurs_dataset_enga = merged_dataset.filter(lambda example: example['text_en'] is not None)

Map:   0%|          | 0/4056 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4056 [00:00<?, ? examples/s]

In [ ]:
fleurs_dataset_enga

Dataset({
    features: ['id', 'audio', 'text_ga', 'text_en'],
    num_rows: 3991
})

In [ ]:
# Check out some samples to make sure it is well-aligned
fleurs_dataset_enga[1000]

{'id': 329,
 'audio': {'path': 'train/15834551979343337701.wav',
  'array': array([0.        , 0.        , 0.        , ..., 0.00057214, 0.0004546 ,
         0.00032961]),
  'sampling_rate': 16000},
 'text_ga': "Chuaigh sé isteach leo i 1945 agus d'fhan sé go dtí 1958.",
 'text_en': 'He joined them in 1945 and stayed until 1958.'}

In [ ]:
# Upload FLEURS-GA-EN dataset to Hugging Face Hub

fleurs_dataset_enga.push_to_hub("ymoslem/FLEURS-GA-EN",
                                data_dir="data",
                                split="train",
                                private=True
                                )